In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
df=pd.read_csv('/content/urldata.csv')
df=df.drop(['Unnamed: 0','label'], axis = 1)
df.shape

(450176, 2)

In [ ]:
def getTokens(input):
    tokensBySlash = str(input.encode('utf-8')).split('/')
    allTokens=[]
    for i in tokensBySlash:
        tokens = str(i).split('-')
        tokensByDot = []
        for j in range(0,len(tokens)):
            tempTokens = str(tokens[j]).split('.')
            tokentsByDot = tokensByDot + tempTokens
        allTokens = allTokens + tokens + tokensByDot
    allTokens = list(set(allTokens))
    if 'com' in allTokens:
        allTokens.remove('com')
    return allTokens

In [ ]:
encoder=HashingVectorizer(n_features=1000,ngram_range=(3,3),tokenizer=getTokens)
test=encoder.fit_transform(df.url.to_numpy())
test

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


<450176x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2072192 stored elements in Compressed Sparse Row format>

In [ ]:
numpyArray=test.toarray()
panda_df = pd.DataFrame(data = numpyArray[0:, 0:],
                        index = ['Row_' + str(i + 1) 
                        for i in range(numpyArray.shape[0])],
                        columns = ['Column_' + str(i + 1) 
                        for i in range(numpyArray.shape[1])])


In [ ]:
panda_df.describe()

KeyboardInterrupt: ignored

In [ ]:
panda_df.to_csv('encoded.csv')


KeyboardInterrupt: ignored

In [ ]:
df['url_length'] = df['url'].apply(lambda i: len(str(i)))
df.head()

,url,result,url_length
0,https://www.google.com,0,22
1,https://www.youtube.com,0,23
2,https://www.facebook.com,0,24
3,https://www.baidu.com,0,21
4,https://www.wikipedia.org,0,25


In [ ]:
from collections import Counter
import itertools

In [ ]:
df['url_schar'] = df['url'].apply(lambda i: len(re.findall(':|_|\?|=|&',i)))
df.head()

,url,result,url_length,url_schar
0,https://www.google.com,0,22,1
1,https://www.youtube.com,0,23,1
2,https://www.facebook.com,0,24,1
3,https://www.baidu.com,0,21,1
4,https://www.wikipedia.org,0,25,1


In [ ]:
def digit_ratio(url):
    digits = 0
    l=0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
        if i.isalpha():
            l=l+1
    return digits/l
df['url_dtl'] = df['url'].apply(lambda i: digit_ratio(i))

In [ ]:
!pip install tld

In [ ]:
df.head()
from urllib.parse import urlparse
from tld import get_tld
import os.path
df['tld'] = df['url'].apply(lambda i: get_tld(i,fail_silently=True))
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
def tld_length(tld):
    try:
        if tld in ['ru','cn','surf','tw','to','gq','ml'] :
          return 1
        else:
          return 0
    except:
        return -1

df['tld_length'] = df['tld'].apply(lambda i: tld_length(i))
df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))
df['hostname_length'] = df['url'].apply(lambda i: len(urlparse(i).netloc))
df.head()

,url,result,url_length,url_schar,url_dtl,tld,tld_length,use_of_ip,hostname_length
0,https://www.google.com,0,22,1,0.0,com,0,0,14
1,https://www.youtube.com,0,23,1,0.0,com,0,0,15
2,https://www.facebook.com,0,24,1,0.0,com,0,0,16
3,https://www.baidu.com,0,21,1,0.0,com,0,0,13
4,https://www.wikipedia.org,0,25,1,0.0,org,0,0,17


In [ ]:
!pip install tldextract

In [ ]:
import tldextract
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits
df['count-digits']= df['url'].apply(lambda i: digit_count(urlparse(i).netloc))
df['hyphen']= df['url'].apply(lambda i: urlparse(i).netloc.count('-'))
df['and']= df['url'].apply(lambda i: urlparse(i).netloc.count('@'))  
df['subdomain']= df['url'].apply(lambda i: tldextract.extract(i).subdomain.count('.'))
df.head()  

,url,result,url_length,url_schar,url_dtl,tld,tld_length,use_of_ip,hostname_length,count-digits,hyphen,and,subdomain
0,https://www.google.com,0,22,1,0.0,com,0,0,14,0,0,0,0
1,https://www.youtube.com,0,23,1,0.0,com,0,0,15,0,0,0,0
2,https://www.facebook.com,0,24,1,0.0,com,0,0,16,0,0,0,0
3,https://www.baidu.com,0,21,1,0.0,com,0,0,13,0,0,0,0
4,https://www.wikipedia.org,0,25,1,0.0,org,0,0,17,0,0,0,0


In [ ]:
def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')
df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')
df['count_embed_domian'] = df['url'].apply(lambda i: no_of_embed(i))

In [ ]:
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('%20')
df['count_20'] = df['url'].apply(lambda i: no_of_embed(i))
df.head()  

,url,result,url_length,url_schar,url_dtl,tld,tld_length,use_of_ip,hostname_length,count-digits,hyphen,and,subdomain,count_dir,count_embed_domian,count_20
0,https://www.google.com,0,22,1,0.0,com,0,0,14,0,0,0,0,0,0,0
1,https://www.youtube.com,0,23,1,0.0,com,0,0,15,0,0,0,0,0,0,0
2,https://www.facebook.com,0,24,1,0.0,com,0,0,16,0,0,0,0,0,0,0
3,https://www.baidu.com,0,21,1,0.0,com,0,0,13,0,0,0,0,0,0,0
4,https://www.wikipedia.org,0,25,1,0.0,org,0,0,17,0,0,0,0,0,0,0


In [ ]:
def spchar(url):
    urldir = urlparse(url).path
    return len(re.findall("[$&+,:;=?@#|'<>.-^*()%!]",urldir))
df['count_spchar'] = df['url'].apply(lambda i: spchar(i))

In [ ]:

df['count_query'] = df['url'].apply(lambda i: len(urlparse(i).query))
df.head()  

,url,result,url_length,url_schar,url_dtl,tld,tld_length,use_of_ip,hostname_length,count-digits,hyphen,and,subdomain,count_dir,count_embed_domian,count_20,count_spchar,count_query
0,https://www.google.com,0,22,1,0.0,com,0,0,14,0,0,0,0,0,0,0,0,0
1,https://www.youtube.com,0,23,1,0.0,com,0,0,15,0,0,0,0,0,0,0,0,0
2,https://www.facebook.com,0,24,1,0.0,com,0,0,16,0,0,0,0,0,0,0,0,0
3,https://www.baidu.com,0,21,1,0.0,com,0,0,13,0,0,0,0,0,0,0,0,0
4,https://www.wikipedia.org,0,25,1,0.0,org,0,0,17,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv('preproc.csv')

In [ ]:
new_df=df.drop(['url','tld'], axis = 1)
new_df.head()

,result,url_length,url_schar,url_dtl,tld_length,use_of_ip,hostname_length,count-digits,hyphen,and,subdomain,count_dir,count_embed_domian,count_20,count_spchar,count_query
0,0,22,1,0.0,0,0,14,0,0,0,0,0,0,0,0,0
1,0,23,1,0.0,0,0,15,0,0,0,0,0,0,0,0,0
2,0,24,1,0.0,0,0,16,0,0,0,0,0,0,0,0,0
3,0,21,1,0.0,0,0,13,0,0,0,0,0,0,0,0,0
4,0,25,1,0.0,0,0,17,0,0,0,0,0,0,0,0,0


In [ ]:
cols = new_df.columns.tolist()
cols = cols[1:] + cols[0:1]
new_df = new_df[cols]
new_df.head()

,url_length,url_schar,url_dtl,tld_length,use_of_ip,hostname_length,count-digits,hyphen,and,subdomain,count_dir,count_embed_domian,count_20,count_spchar,count_query,result
0,22,1,0.0,0,0,14,0,0,0,0,0,0,0,0,0,0
1,23,1,0.0,0,0,15,0,0,0,0,0,0,0,0,0,0
2,24,1,0.0,0,0,16,0,0,0,0,0,0,0,0,0,0
3,21,1,0.0,0,0,13,0,0,0,0,0,0,0,0,0,0
4,25,1,0.0,0,0,17,0,0,0,0,0,0,0,0,0,0


In [ ]:
import os
import pickle
def byHDF(dfs):
    store=pd.HDFStore('df_all.h5')
    for df in dfs:
        store.append('df',df,data_columns=list('0123'))
    #del dfs
    df=store.select('df')
    store.close()
    os.remove('df_all.h5')
    return df

In [ ]:
new_df.to_csv('preproc.csv')

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
train= pd.read_csv('/gdrive/MyDrive/ml/encoded.csv')

In [ ]:
train = train.dropna(axis = 0, how ='any')

In [ ]:
from sklearn.model_selection import train_test_split


"labels = np.array(train['result'])\nfeatures= train.drop('result', axis = 1)\n# Saving feature names for later use\nfeature_list = list(features.columns)\n# Convert to numpy array\nfeatures = np.array(features)"

In [ ]:
del [train]

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(train.drop('result', axis = 1), train.result, test_size = 0.25, random_state = 42)

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



In [ ]:
classifier = XGBClassifier(tree_method='gpu_hist')
model = classifier.fit(train_features, train_labels)

In [ ]:
model.save_model('model_file_name.json')

In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('model_file_name.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new=classifier.load_model('/content/model_file_name.json')

In [ ]:
import pickle
pickle.dump(model, open("trained.dat", "wb"))

In [ ]:
y_pred=classifier.predict(test_features)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, y_pred)

In [ ]:
accuracy

0.8885392576866209

In [ ]:
loaded_model = pickle.load(open("/content/trained.dat", "rb"))
# make predictions for test data
y_predl = loaded_model.predict(test_features)

# evaluate predictions
accuracy = accuracy_score(test_labels, y_predl)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.85%
